In [77]:
import joblib
import pickle
import torch

#Basma
# classifer = joblib.load("C:/Users/basma/Fourth Year/Graduation_Project/squeezenetfinetuned.pkl")

#Bassem
classifer = joblib.load("squeezenetuntrained.pkl")

In [78]:
# loaded_model = pickle.load("squeezenetfinetuned.pkl")
import sklearn 
import numpy as np

#Basma
# model = pickle.load(open('C:/Users/basma/Fourth Year/Graduation_Project/squeezenetuntrained.pkl', 'rb'))

#Bassem
model = pickle.load(open('squeezenetuntrained.pkl', 'rb'))
model.eval()

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

In [79]:
import torchvision.transforms as transforms
from PIL import Image
from PIL import ImageFile
import cv2

#Basma
# input_image = Image.open('C:/Users/basma/Fourth Year/Graduation_Project/SegmentedDataset/test/O001/O001_1.png').convert('RGB')
#Bassem
input_image = Image.open('D:/Ahmed Bassem/MIU/Year 4/Graduation Project/Datasets/SegmentedDataset/SegmentedDataset/test/O001/O001_1.png').convert('RGB')


preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)

# Tensor of shape 1000, with confidence scores over Kemet's 1072 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)

tensor([ 2.8720,  2.4633,  3.9257,  ..., 17.4090, 13.0940, 14.6272])


c:\Users\Ahmed B\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


In [80]:
# Read the categories

#Basma
# with open("C:/Users/basma/Fourth Year/Graduation_Project/classes_names.txt", "r") as f:

#Bassem
with open("D:/Ahmed Bassem/MIU\Year 4/Graduation Project/classes_names.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

top1 = categories[top5_catid[0]]
print(top1)

O001 0.9966236352920532
O005 0.0027625213842839003
O004 0.00042799804941751063
O017 4.750968946609646e-05
O006E 3.333542917971499e-05
O001


In [81]:
import matching_algorithm as ma
print(ma.dictionary_matching(top1))


House


In [82]:
import googletrans
from googletrans import *

In [83]:
match = ma.dictionary_matching(top1)

In [84]:
translator = googletrans.Translator()
translate=translator.translate(match,dest='arabic')
print(translate.text)

منزل
